# Análisis Estadístico Profundo - Calidad de Servicio SERVQUAL
## Fundación Teletón - Diagnóstico de Áreas de Oportunidad

Este notebook realiza un análisis estadístico riguroso para identificar:
- **Drivers de satisfacción**: ¿Qué dimensiones SERVQUAL impactan más el NPS?
- **Diferencias significativas**: ¿Hay diferencias por tipo de organización, región o antigüedad?
- **Segmentos críticos**: ¿Dónde están los benefactores insatisfechos?
- **Áreas de oportunidad**: Recomendaciones basadas en evidencia estadística

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, f_oneway, pearsonr, spearmanr
from scipy.stats import shapiro, levene
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

# Configuración visual
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Librerías cargadas correctamente")

In [ ]:
# Cargar datos enriquecidos
df = pd.read_csv('../data/teleton_enriched.csv', encoding='utf-8-sig')
print(f"Dataset: {df.shape[0]} registros x {df.shape[1]} columnas")
df.head()

In [ ]:
# Definir variables
vars_servqual = ['AT_1', 'AT_2', 'FI_1', 'FI_2', 'FI_3', 'R_1', 'R_2', 'R_3', 'E_1', 'E_2', 'E_3', 'E_4']
vars_scores = ['score_tangibles', 'score_fiabilidad', 'score_responsiveness', 'score_empatia']
vars_outcome = ['D_1', 'NPS', 'C_1', 'INFO']
vars_categoricas = ['Giro', 'Puesto', 'region', 'antiguedad_grupo', 'nps_categoria', 'calidad_nivel']

---
# 1. DIAGNÓSTICO INICIAL - Visión del Analista

## 1.1 Métricas Clave de Calidad de Servicio

In [ ]:
# Resumen ejecutivo de métricas
print("="*70)
print("DIAGNÓSTICO DE CALIDAD DE SERVICIO - FUNDACIÓN TELETÓN")
print("="*70)

# NPS Score
nps_counts = df['nps_categoria'].value_counts(normalize=True) * 100
nps_score = nps_counts.get('Promotor', 0) - nps_counts.get('Detractor', 0)

print(f"\n📊 MÉTRICAS GENERALES")
print(f"   NPS Score: {nps_score:.1f}")
print(f"   Satisfacción promedio: {df['D_1'].mean():.2f}/10")
print(f"   Calidad percibida: {df['C_1'].mean():.2f}/5")
print(f"   Score SERVQUAL total: {df['score_servqual_total'].mean():.2f}/5")

print(f"\n📈 DISTRIBUCIÓN NPS")
print(f"   Promotores (9-10): {nps_counts.get('Promotor', 0):.1f}%")
print(f"   Pasivos (7-8): {nps_counts.get('Pasivo', 0):.1f}%")
print(f"   Detractores (1-6): {nps_counts.get('Detractor', 0):.1f}%")

In [ ]:
# Scores por dimensión SERVQUAL
print("\n📋 SCORES POR DIMENSIÓN SERVQUAL")
print("-"*40)
for score in vars_scores:
    dim_name = score.replace('score_', '').upper()
    mean_val = df[score].mean()
    std_val = df[score].std()
    status = "✅" if mean_val >= 4 else "⚠️" if mean_val >= 3.5 else "❌"
    print(f"   {dim_name:15s}: {mean_val:.2f} ± {std_val:.2f} {status}")

## 1.2 Identificación de Segmentos Críticos

In [ ]:
# Identificar benefactores insatisfechos (Detractores)
detractores = df[df['nps_categoria'] == 'Detractor']
promotores = df[df['nps_categoria'] == 'Promotor']

print("\n🔴 ANÁLISIS DE DETRACTORES (benefactores insatisfechos)")
print("="*60)
print(f"Total detractores: {len(detractores)} ({len(detractores)/len(df)*100:.1f}%)")

if len(detractores) > 0:
    print(f"\nPerfil de detractores:")
    print(f"   Giro más común: {detractores['Giro'].mode().values[0] if len(detractores['Giro'].mode()) > 0 else 'N/A'}")
    print(f"   Región más común: {detractores['region'].mode().values[0] if len(detractores['region'].dropna().mode()) > 0 else 'N/A'}")
    print(f"   Antigüedad promedio: {detractores['AÑOS'].mean():.1f} años")
    
    print(f"\nScores SERVQUAL de detractores vs promotores:")
    for score in vars_scores:
        det_mean = detractores[score].mean()
        prom_mean = promotores[score].mean()
        diff = prom_mean - det_mean
        print(f"   {score.replace('score_', ''):15s}: Detractores={det_mean:.2f}, Promotores={prom_mean:.2f} (gap={diff:.2f})")

In [ ]:
# Análisis por Giro - identificar segmentos problemáticos
print("\n📊 SATISFACCIÓN POR TIPO DE ORGANIZACIÓN")
print("="*60)

giro_stats = df.groupby('Giro').agg({
    'D_1': ['mean', 'std', 'count'],
    'NPS': 'mean',
    'score_servqual_total': 'mean'
}).round(2)

giro_stats.columns = ['Satisfacción', 'Std', 'N', 'NPS', 'SERVQUAL']
giro_stats = giro_stats.sort_values('Satisfacción', ascending=False)

print(giro_stats.to_string())

# Identificar giro con menor satisfacción
giro_peor = giro_stats['Satisfacción'].idxmin()
print(f"\n⚠️ ÁREA DE OPORTUNIDAD: '{giro_peor}' tiene la menor satisfacción promedio")

In [ ]:
# Análisis por Región
print("\n📍 SATISFACCIÓN POR REGIÓN GEOGRÁFICA")
print("="*60)

region_stats = df.groupby('region').agg({
    'D_1': ['mean', 'count'],
    'NPS': 'mean',
    'score_servqual_total': 'mean'
}).round(2)

region_stats.columns = ['Satisfacción', 'N', 'NPS', 'SERVQUAL']
region_stats = region_stats.sort_values('Satisfacción', ascending=False)

print(region_stats.to_string())

---
# 2. ANÁLISIS DE CORRELACIONES

## 2.1 Correlación de Pearson - Variables Numéricas

In [ ]:
# Matriz de correlación completa
vars_numericas = vars_scores + vars_outcome + ['AÑOS', 'score_servqual_total']
corr_matrix = df[vars_numericas].corr()

# Visualización
fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r',
            center=0, vmin=-1, vmax=1, ax=ax, square=True)
ax.set_title('Matriz de Correlaciones - Variables Numéricas', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Correlaciones específicas con NPS (variable objetivo)
print("\n🎯 CORRELACIONES CON NPS (variable objetivo)")
print("="*60)

correlaciones_nps = []
for var in vars_scores + ['D_1', 'C_1', 'INFO', 'AÑOS']:
    corr, p_value = pearsonr(df[var].dropna(), df.loc[df[var].notna(), 'NPS'])
    sig = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else ""
    correlaciones_nps.append({'Variable': var, 'Correlación': corr, 'p-value': p_value, 'Sig': sig})

corr_df = pd.DataFrame(correlaciones_nps).sort_values('Correlación', ascending=False)
print(corr_df.to_string(index=False))

print("\nNota: *** p<0.001, ** p<0.01, * p<0.05")

In [ ]:
# Visualizar correlaciones con NPS
fig, ax = plt.subplots(figsize=(10, 6))

corr_nps = df[vars_scores].corrwith(df['NPS']).sort_values(ascending=True)
colors = ['#e74c3c' if x < 0.3 else '#f39c12' if x < 0.5 else '#27ae60' for x in corr_nps.values]

bars = ax.barh([s.replace('score_', '').capitalize() for s in corr_nps.index], 
               corr_nps.values, color=colors)
ax.axvline(x=0.3, color='orange', linestyle='--', alpha=0.7, label='Correlación débil')
ax.axvline(x=0.5, color='green', linestyle='--', alpha=0.7, label='Correlación moderada')

for bar, val in zip(bars, corr_nps.values):
    ax.text(val + 0.02, bar.get_y() + bar.get_height()/2, f'{val:.3f}', va='center', fontsize=11)

ax.set_xlabel('Correlación con NPS')
ax.set_title('¿Qué dimensión SERVQUAL impacta más el NPS?', fontweight='bold', fontsize=14)
ax.legend(loc='lower right')
ax.set_xlim(0, 0.8)
plt.tight_layout()
plt.show()

dim_mas_importante = corr_nps.idxmax().replace('score_', '').upper()
print(f"\n💡 INSIGHT: La dimensión '{dim_mas_importante}' es la que más impacta el NPS")

---
# 3. PRUEBAS DE HIPÓTESIS

## 3.1 Prueba Chi-Cuadrada - Asociación entre Variables Categóricas

In [ ]:
def chi_square_test(df, var1, var2):
    """Realiza prueba chi-cuadrada y retorna resultados"""
    contingency = pd.crosstab(df[var1], df[var2])
    chi2, p_value, dof, expected = chi2_contingency(contingency)
    
    # Cramér's V para tamaño del efecto
    n = contingency.sum().sum()
    min_dim = min(contingency.shape) - 1
    cramers_v = np.sqrt(chi2 / (n * min_dim)) if min_dim > 0 else 0
    
    return {
        'chi2': chi2,
        'p_value': p_value,
        'dof': dof,
        'cramers_v': cramers_v,
        'significativo': p_value < 0.05
    }

In [ ]:
# Chi-cuadrada: NPS_categoria vs Giro
print("\n🔬 PRUEBA CHI-CUADRADA: NPS Categoría vs Tipo de Organización")
print("="*70)
print("H₀: No hay asociación entre el tipo de organización y la categoría NPS")
print("H₁: Existe asociación entre el tipo de organización y la categoría NPS")
print("-"*70)

result = chi_square_test(df, 'nps_categoria', 'Giro')
print(f"\nResultados:")
print(f"   Chi² = {result['chi2']:.2f}")
print(f"   Grados de libertad = {result['dof']}")
print(f"   p-value = {result['p_value']:.4f}")
print(f"   Cramér's V = {result['cramers_v']:.3f} (tamaño del efecto)")

if result['significativo']:
    print(f"\n✅ CONCLUSIÓN: Se RECHAZA H₀ (p < 0.05)")
    print(f"   Existe asociación significativa entre el tipo de organización y el NPS.")
else:
    print(f"\n❌ CONCLUSIÓN: NO se rechaza H₀ (p >= 0.05)")
    print(f"   No hay evidencia de asociación entre tipo de organización y NPS.")

In [ ]:
# Tabla de contingencia visual
contingency = pd.crosstab(df['Giro'], df['nps_categoria'], normalize='index') * 100

fig, ax = plt.subplots(figsize=(10, 6))
contingency.plot(kind='barh', stacked=True, ax=ax, 
                 color=['#e74c3c', '#f39c12', '#27ae60'])
ax.set_xlabel('Porcentaje')
ax.set_title('Distribución de NPS por Tipo de Organización', fontweight='bold')
ax.legend(title='NPS Categoría', bbox_to_anchor=(1.02, 1))
plt.tight_layout()
plt.show()

In [ ]:
# Chi-cuadrada: NPS_categoria vs Antigüedad
print("\n🔬 PRUEBA CHI-CUADRADA: NPS Categoría vs Antigüedad del Benefactor")
print("="*70)

result2 = chi_square_test(df.dropna(subset=['antiguedad_grupo']), 'nps_categoria', 'antiguedad_grupo')
print(f"   Chi² = {result2['chi2']:.2f}, p-value = {result2['p_value']:.4f}")
print(f"   {'✅ Significativo' if result2['significativo'] else '❌ No significativo'}")

In [ ]:
# Chi-cuadrada: NPS_categoria vs Región
print("\n🔬 PRUEBA CHI-CUADRADA: NPS Categoría vs Región")
print("="*70)

df_region = df.dropna(subset=['region'])
if len(df_region['region'].unique()) > 1:
    result3 = chi_square_test(df_region, 'nps_categoria', 'region')
    print(f"   Chi² = {result3['chi2']:.2f}, p-value = {result3['p_value']:.4f}")
    print(f"   {'✅ Significativo' if result3['significativo'] else '❌ No significativo'}")

## 3.2 Prueba t de Student - Comparación de Dos Grupos

In [ ]:
def t_test_analysis(group1, group2, name1, name2, variable):
    """Realiza prueba t y retorna análisis completo"""
    # Prueba de normalidad (Shapiro-Wilk)
    if len(group1) >= 3 and len(group2) >= 3:
        _, p_norm1 = shapiro(group1[:50])  # Muestra para Shapiro
        _, p_norm2 = shapiro(group2[:50])
    else:
        p_norm1, p_norm2 = 1, 1
    
    # Prueba de homogeneidad de varianzas (Levene)
    _, p_levene = levene(group1, group2)
    
    # Prueba t (Welch si varianzas desiguales)
    equal_var = p_levene > 0.05
    t_stat, p_value = ttest_ind(group1, group2, equal_var=equal_var)
    
    # Tamaño del efecto (Cohen's d)
    pooled_std = np.sqrt(((len(group1)-1)*group1.std()**2 + (len(group2)-1)*group2.std()**2) / 
                         (len(group1)+len(group2)-2))
    cohens_d = (group1.mean() - group2.mean()) / pooled_std if pooled_std > 0 else 0
    
    return {
        't_stat': t_stat,
        'p_value': p_value,
        'cohens_d': cohens_d,
        'mean1': group1.mean(),
        'mean2': group2.mean(),
        'n1': len(group1),
        'n2': len(group2),
        'equal_var': equal_var
    }

In [ ]:
# Prueba t: Empresas vs Educación (los dos giros más grandes)
print("\n🔬 PRUEBA T: Satisfacción de Empresas vs Instituciones Educativas")
print("="*70)
print("H₀: μ_empresas = μ_educación (no hay diferencia en satisfacción)")
print("H₁: μ_empresas ≠ μ_educación (hay diferencia en satisfacción)")
print("-"*70)

empresas = df[df['Giro'] == 'Empresa']['D_1'].dropna()
educacion = df[df['Giro'] == 'Educación']['D_1'].dropna()

if len(empresas) > 5 and len(educacion) > 5:
    result_t = t_test_analysis(empresas, educacion, 'Empresas', 'Educación', 'D_1')
    
    print(f"\nResultados:")
    print(f"   Empresas: M = {result_t['mean1']:.2f}, n = {result_t['n1']}")
    print(f"   Educación: M = {result_t['mean2']:.2f}, n = {result_t['n2']}")
    print(f"   t = {result_t['t_stat']:.3f}")
    print(f"   p-value = {result_t['p_value']:.4f}")
    print(f"   Cohen's d = {result_t['cohens_d']:.3f} (tamaño del efecto)")
    
    effect_size = "pequeño" if abs(result_t['cohens_d']) < 0.5 else "mediano" if abs(result_t['cohens_d']) < 0.8 else "grande"
    
    if result_t['p_value'] < 0.05:
        print(f"\n✅ CONCLUSIÓN: Diferencia SIGNIFICATIVA (p < 0.05)")
        print(f"   Efecto {effect_size}. {'Empresas' if result_t['mean1'] > result_t['mean2'] else 'Educación'} tiene mayor satisfacción.")
    else:
        print(f"\n❌ CONCLUSIÓN: NO hay diferencia significativa (p >= 0.05)")

In [ ]:
# Prueba t: Nuevos vs Veteranos
print("\n🔬 PRUEBA T: Satisfacción de Benefactores Nuevos vs Veteranos")
print("="*70)

nuevos = df[df['antiguedad_grupo'] == 'Nuevo']['D_1'].dropna()
veteranos = df[df['antiguedad_grupo'] == 'Veterano']['D_1'].dropna()

if len(nuevos) > 5 and len(veteranos) > 5:
    result_t2 = t_test_analysis(nuevos, veteranos, 'Nuevos', 'Veteranos', 'D_1')
    
    print(f"   Nuevos: M = {result_t2['mean1']:.2f}, n = {result_t2['n1']}")
    print(f"   Veteranos: M = {result_t2['mean2']:.2f}, n = {result_t2['n2']}")
    print(f"   t = {result_t2['t_stat']:.3f}, p-value = {result_t2['p_value']:.4f}")
    print(f"   Cohen's d = {result_t2['cohens_d']:.3f}")
    print(f"   {'✅ Significativo' if result_t2['p_value'] < 0.05 else '❌ No significativo'}")

In [ ]:
# Prueba t: Promotores vs Detractores en Score SERVQUAL
print("\n🔬 PRUEBA T: Score SERVQUAL de Promotores vs Detractores")
print("="*70)

promotores_servqual = df[df['nps_categoria'] == 'Promotor']['score_servqual_total'].dropna()
detractores_servqual = df[df['nps_categoria'] == 'Detractor']['score_servqual_total'].dropna()

if len(promotores_servqual) > 5 and len(detractores_servqual) > 5:
    result_t3 = t_test_analysis(promotores_servqual, detractores_servqual, 'Promotores', 'Detractores', 'SERVQUAL')
    
    print(f"   Promotores: M = {result_t3['mean1']:.2f}, n = {result_t3['n1']}")
    print(f"   Detractores: M = {result_t3['mean2']:.2f}, n = {result_t3['n2']}")
    print(f"   t = {result_t3['t_stat']:.3f}, p-value = {result_t3['p_value']:.6f}")
    print(f"   Cohen's d = {result_t3['cohens_d']:.3f}")
    
    if result_t3['p_value'] < 0.05:
        print(f"\n💡 INSIGHT: Los promotores perciben significativamente mejor calidad de servicio")
        print(f"   Gap de {result_t3['mean1'] - result_t3['mean2']:.2f} puntos en escala SERVQUAL")

## 3.3 ANOVA - Comparación de Múltiples Grupos

In [ ]:
# ANOVA: Satisfacción por Tipo de Organización (Giro)
print("\n🔬 ANOVA: Satisfacción por Tipo de Organización")
print("="*70)
print("H₀: μ₁ = μ₂ = ... = μₖ (todas las medias son iguales)")
print("H₁: Al menos una media es diferente")
print("-"*70)

# Preparar grupos
grupos = [df[df['Giro'] == giro]['D_1'].dropna() for giro in df['Giro'].unique()]
grupos = [g for g in grupos if len(g) >= 5]  # Solo grupos con n >= 5

if len(grupos) >= 2:
    f_stat, p_value = f_oneway(*grupos)
    
    # Calcular eta-squared (tamaño del efecto)
    df_anova = df[['Giro', 'D_1']].dropna()
    ss_between = sum(len(g) * (g.mean() - df_anova['D_1'].mean())**2 for g in grupos)
    ss_total = sum((df_anova['D_1'] - df_anova['D_1'].mean())**2)
    eta_squared = ss_between / ss_total if ss_total > 0 else 0
    
    print(f"\nResultados:")
    print(f"   F = {f_stat:.3f}")
    print(f"   p-value = {p_value:.4f}")
    print(f"   η² = {eta_squared:.3f} (tamaño del efecto)")
    
    if p_value < 0.05:
        print(f"\n✅ CONCLUSIÓN: Existen diferencias SIGNIFICATIVAS entre grupos")
        print(f"   Se recomienda análisis post-hoc para identificar qué grupos difieren.")
    else:
        print(f"\n❌ CONCLUSIÓN: NO hay diferencias significativas entre grupos")

In [ ]:
# Post-hoc Tukey HSD (si ANOVA es significativo)
if p_value < 0.05:
    print("\n📊 ANÁLISIS POST-HOC: Tukey HSD")
    print("="*70)
    
    df_tukey = df[['Giro', 'D_1']].dropna()
    tukey = pairwise_tukeyhsd(df_tukey['D_1'], df_tukey['Giro'], alpha=0.05)
    print(tukey)
    
    # Identificar diferencias significativas
    print("\n💡 Pares con diferencias significativas:")
    tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
    sig_pairs = tukey_df[tukey_df['reject'] == True]
    if len(sig_pairs) > 0:
        for _, row in sig_pairs.iterrows():
            print(f"   {row['group1']} vs {row['group2']}: diferencia = {row['meandiff']:.2f}")
    else:
        print("   No se encontraron diferencias significativas entre pares específicos")

In [ ]:
# Visualización ANOVA - Boxplot por grupo
fig, ax = plt.subplots(figsize=(12, 6))

# Ordenar por media
orden = df.groupby('Giro')['D_1'].mean().sort_values(ascending=False).index

df.boxplot(column='D_1', by='Giro', ax=ax, positions=range(len(orden)))
ax.set_xticklabels(orden, rotation=45, ha='right')
ax.axhline(y=df['D_1'].mean(), color='red', linestyle='--', label=f'Media general: {df["D_1"].mean():.1f}')
ax.set_xlabel('Tipo de Organización')
ax.set_ylabel('Satisfacción (1-10)')
ax.set_title('Distribución de Satisfacción por Tipo de Organización', fontweight='bold')
ax.legend()
plt.suptitle('')
plt.tight_layout()
plt.show()

In [ ]:
# ANOVA: NPS por Antigüedad
print("\n🔬 ANOVA: NPS por Antigüedad del Benefactor")
print("="*70)

df_antig = df.dropna(subset=['antiguedad_grupo', 'NPS'])
grupos_antig = [df_antig[df_antig['antiguedad_grupo'] == g]['NPS'] for g in ['Nuevo', 'Establecido', 'Veterano']]
grupos_antig = [g for g in grupos_antig if len(g) >= 5]

if len(grupos_antig) >= 2:
    f_stat2, p_value2 = f_oneway(*grupos_antig)
    print(f"   F = {f_stat2:.3f}, p-value = {p_value2:.4f}")
    print(f"   {'✅ Significativo' if p_value2 < 0.05 else '❌ No significativo'}")

---
# 4. REGRESIÓN LINEAL MÚLTIPLE

## 4.1 Modelo: Predecir NPS con Dimensiones SERVQUAL

In [ ]:
# Preparar datos para regresión
df_reg = df[vars_scores + ['NPS', 'AÑOS']].dropna()

X = df_reg[vars_scores]
y = df_reg['NPS']

# Agregar constante
X_const = sm.add_constant(X)

# Ajustar modelo OLS
modelo = sm.OLS(y, X_const).fit()

print("\n📈 REGRESIÓN LINEAL MÚLTIPLE: Predicción de NPS")
print("="*70)
print(modelo.summary())

In [ ]:
# Interpretación del modelo
print("\n💡 INTERPRETACIÓN DEL MODELO DE REGRESIÓN")
print("="*70)

print(f"\n📊 Bondad de ajuste:")
print(f"   R² = {modelo.rsquared:.3f} ({modelo.rsquared*100:.1f}% de varianza explicada)")
print(f"   R² ajustado = {modelo.rsquared_adj:.3f}")
print(f"   F-estadístico = {modelo.fvalue:.2f} (p = {modelo.f_pvalue:.4f})")

print(f"\n📋 Coeficientes (impacto en NPS):")
for var in vars_scores:
    coef = modelo.params[var]
    p_val = modelo.pvalues[var]
    sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
    print(f"   {var.replace('score_', ''):15s}: β = {coef:+.3f} (p = {p_val:.4f}) {sig}")

# Variable más importante
coefs_abs = modelo.params[vars_scores].abs()
var_mas_importante = coefs_abs.idxmax()
print(f"\n🎯 VARIABLE MÁS IMPORTANTE: {var_mas_importante.replace('score_', '').upper()}")
print(f"   Un aumento de 1 punto en {var_mas_importante.replace('score_', '')} se asocia con")
print(f"   un cambio de {modelo.params[var_mas_importante]:+.2f} puntos en NPS.")

In [ ]:
# Visualización de coeficientes
fig, ax = plt.subplots(figsize=(10, 6))

coefs = modelo.params[vars_scores].sort_values()
colors = ['#27ae60' if c > 0 else '#e74c3c' for c in coefs.values]

bars = ax.barh([s.replace('score_', '').capitalize() for s in coefs.index], 
               coefs.values, color=colors)
ax.axvline(x=0, color='black', linewidth=0.5)

for bar, val, var in zip(bars, coefs.values, coefs.index):
    p_val = modelo.pvalues[var]
    sig = "*" if p_val < 0.05 else ""
    ax.text(val + 0.05 if val > 0 else val - 0.15, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}{sig}', va='center', fontsize=11)

ax.set_xlabel('Coeficiente de Regresión')
ax.set_title('Impacto de cada Dimensión SERVQUAL en el NPS', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Verificar multicolinealidad (VIF)
print("\n🔍 DIAGNÓSTICO DE MULTICOLINEALIDAD (VIF)")
print("="*50)

vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data.to_string(index=False))
print("\nInterpretación: VIF > 5 indica multicolinealidad problemática")

if any(vif_data['VIF'] > 5):
    print("⚠️ ADVERTENCIA: Algunas variables tienen alta multicolinealidad")
else:
    print("✅ No hay problemas de multicolinealidad")

In [ ]:
# Modelo 2: Predecir Satisfacción (D_1)
print("\n📈 REGRESIÓN LINEAL: Predicción de Satisfacción (D_1)")
print("="*70)

df_reg2 = df[vars_scores + ['D_1']].dropna()
X2 = df_reg2[vars_scores]
y2 = df_reg2['D_1']
X2_const = sm.add_constant(X2)

modelo2 = sm.OLS(y2, X2_const).fit()

print(f"R² = {modelo2.rsquared:.3f} ({modelo2.rsquared*100:.1f}% varianza explicada)")
print(f"\nCoeficientes significativos:")
for var in vars_scores:
    if modelo2.pvalues[var] < 0.05:
        print(f"   {var.replace('score_', ''):15s}: β = {modelo2.params[var]:+.3f}")

---
# 5. RESUMEN DE HALLAZGOS Y RECOMENDACIONES

In [ ]:
# Generar resumen ejecutivo
print("\n" + "="*80)
print("RESUMEN EJECUTIVO - DIAGNÓSTICO DE CALIDAD DE SERVICIO")
print("="*80)

print("\n📊 1. MÉTRICAS GENERALES")
print(f"   • NPS Score: {nps_score:.0f} {'(Bueno)' if nps_score > 0 else '(Necesita mejora)'}")
print(f"   • Satisfacción: {df['D_1'].mean():.1f}/10")
print(f"   • Score SERVQUAL: {df['score_servqual_total'].mean():.2f}/5")

print("\n🔬 2. HALLAZGOS ESTADÍSTICOS")

# Correlación más alta con NPS
corr_max = corr_df.iloc[0]
print(f"   • La dimensión '{corr_max['Variable'].replace('score_', '')}' tiene la mayor correlación con NPS (r={corr_max['Correlación']:.2f})")

# Chi-cuadrada
if result['significativo']:
    print(f"   • Existe asociación significativa entre tipo de organización y categoría NPS")
else:
    print(f"   • No hay asociación significativa entre tipo de organización y NPS")

# Regresión
print(f"   • El modelo SERVQUAL explica {modelo.rsquared*100:.0f}% de la varianza del NPS")
print(f"   • Variable más predictiva: {var_mas_importante.replace('score_', '').upper()}")

print("\n⚠️ 3. ÁREAS DE OPORTUNIDAD")
dim_scores = {dim: df[f'score_{dim}'].mean() for dim in ['tangibles', 'fiabilidad', 'responsiveness', 'empatia']}
dim_min = min(dim_scores, key=dim_scores.get)
print(f"   • Dimensión con menor score: {dim_min.upper()} ({dim_scores[dim_min]:.2f})")
print(f"   • Segmento con menor satisfacción: {giro_peor}")
print(f"   • {len(detractores)} benefactores detractores ({len(detractores)/len(df)*100:.1f}%) requieren atención")

print("\n💡 4. RECOMENDACIONES")
print(f"   1. Priorizar mejoras en la dimensión '{dim_min.upper()}' - mayor impacto potencial")
print(f"   2. Diseñar estrategia específica para '{giro_peor}'")
print(f"   3. Implementar programa de recuperación para detractores")
print(f"   4. Capacitar promotores en las competencias de '{var_mas_importante.replace('score_', '')}'")

In [ ]:
# Guardar resultados clave para el dashboard
resultados = {
    'nps_score': nps_score,
    'satisfaccion_promedio': df['D_1'].mean(),
    'calidad_promedio': df['C_1'].mean(),
    'servqual_total': df['score_servqual_total'].mean(),
    'dimension_mas_correlacion': corr_max['Variable'].replace('score_', ''),
    'correlacion_max': corr_max['Correlación'],
    'r_squared_modelo': modelo.rsquared,
    'variable_mas_importante': var_mas_importante.replace('score_', ''),
    'dimension_menor_score': dim_min,
    'giro_menor_satisfaccion': giro_peor,
    'n_detractores': len(detractores),
    'pct_detractores': len(detractores)/len(df)*100
}

print("\n📁 Resultados guardados para el dashboard:")
for k, v in resultados.items():
    print(f"   {k}: {v}")